In [1]:
import os

In [2]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow/research'

In [3]:

os.chdir("../")

In [4]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "http://dagshub.com/sripathikoushik244/MLProject-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "sripathikoushik244"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7b130b0c5fe6cc004b637b8cea9c46d8d1470d2f"


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_uri: str

In [8]:
from MLProject_mlflow.constants import *
from MLProject_mlflow.utils.common import create_directories, read_yaml, save_json

In [20]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath= CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_path=Path(config.metric_file_path),
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/sripathikoushik244/MLProject-mlflow.mlflow"
        )

        return model_evaluation_config


In [21]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib
import numpy as np

In [22]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            #Model Registry does not work with file store
            if tracking_url_type_store != "file":

                #Register the model
                mlflow.sklearn.log_model(model, "model", registered_model_name="Elasticnetmodel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [23]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

2025-12-19 09:37:20,400: mlprojectlog: INFO: yaml file: config/config.yaml loaded successfully
2025-12-19 09:37:20,402: mlprojectlog: INFO: yaml file: params.yaml loaded successfully
2025-12-19 09:37:20,406: mlprojectlog: INFO: yaml file: schema.yaml loaded successfully
2025-12-19 09:37:20,407: mlprojectlog: INFO: created directory at path: artifacts
2025-12-19 09:37:20,409: mlprojectlog: INFO: created directory at path: artifacts/model_evaluation
2025-12-19 09:37:20,869: mlprojectlog: INFO: json file saved at: artifacts/model_evaluation/metric.json


2025/12/19 09:37:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'Elasticnetmodel'.
2025/12/19 09:37:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Elasticnetmodel, version 1
Created version '1' of model 'Elasticnetmodel'.


🏃 View run marvelous-cat-465 at: http://dagshub.com/sripathikoushik244/MLProject-mlflow.mlflow/#/experiments/0/runs/357d43796a904887b0cb1f95f630497f
🧪 View experiment at: http://dagshub.com/sripathikoushik244/MLProject-mlflow.mlflow/#/experiments/0
